# 01 - Data Acquisition

Download the LendingClub dataset from Kaggle and perform initial exploration.

**Dataset**: [LendingClub Loan Data](https://www.kaggle.com/datasets/wordsforthewise/lending-club)

**Contents**:

- Download data using kagglehub
- Initial exploration
- Save as parquet for faster reloads


In [1]:
import kagglehub
import pandas as pd
import numpy as np
from pathlib import Path
import warnings

warnings.filterwarnings("ignore")

In [2]:
# Configure paths
DATA_DIR = Path("../data/raw")
DATA_DIR.mkdir(parents=True, exist_ok=True)

print(f"Data directory: {DATA_DIR.resolve()}")

Data directory: /Users/hussain/data/raw


## Download Dataset

Using kagglehub for programmatic download. Requires Kaggle API credentials.

**Setup**: Create `~/.kaggle/kaggle.json` with your API key from https://www.kaggle.com/settings


In [3]:
# Download LendingClub dataset
path = kagglehub.dataset_download("wordsforthewise/lending-club")
print(f"Dataset downloaded to: {path}")

100%|██████████| 1.26G/1.26G [00:29<00:00, 46.1MB/s]

Extracting files...


Dataset downloaded to: /Users/hussain/.cache/kagglehub/datasets/wordsforthewise/lending-club/versions/3


In [4]:
# List downloaded files
import os

files = os.listdir(path)
print("Downloaded files:")
for f in files:
    size_mb = os.path.getsize(os.path.join(path, f)) / (1024 * 1024)
    print(f"  - {f} ({size_mb:.1f} MB)")

Downloaded files:
  - rejected_2007_to_2018q4.csv (0.0 MB)
  - accepted_2007_to_2018q4.csv (0.0 MB)
  - rejected_2007_to_2018Q4.csv.gz (243.6 MB)
  - accepted_2007_to_2018Q4.csv.gz (374.4 MB)


## Load and Explore Data

- Expected time: 5-10 minutes on a MacBook Air, depending on RAM.


In [5]:
# Load the accepted loans dataset (the main one)
# Try different possible filenames
possible_files = [
    "accepted_2007_to_2018Q4.csv.gz",
    "accepted_2007_to_2018Q4.csv",
    "lending_club_loan_two.csv",
]

df = None
for filename in possible_files:
    filepath = os.path.join(path, filename)
    if os.path.exists(filepath):
        print(f"Loading {filename}...")
        df = pd.read_csv(filepath, low_memory=False)
        break

if df is None:
    # Fallback: load any CSV file
    csv_files = [f for f in files if f.endswith(".csv") or f.endswith(".csv.gz")]
    if csv_files:
        filepath = os.path.join(path, csv_files[0])
        print(f"Loading {csv_files[0]}...")
        df = pd.read_csv(filepath, low_memory=False)

print(f"\nDataset loaded successfully!")
print(f"Shape: {df.shape[0]:,} rows x {df.shape[1]} columns")

Loading accepted_2007_to_2018Q4.csv.gz...

Dataset loaded successfully!
Shape: 2,260,701 rows x 151 columns


In [6]:
# Basic info
print("Column names:")
print(df.columns.tolist())

Column names:
['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose', 'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'fico_range_low', 'fico_range_high', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d', 'last_fico_range_high', 'last_fico_range_low', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'ac

In [7]:
# Data types and memory usage
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2260701 entries, 0 to 2260700
Columns: 151 entries, id to settlement_term
dtypes: float64(113), object(38)
memory usage: 2.5+ GB


In [8]:
# Statistical summary
df.describe()

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,fico_range_low,...,deferral_term,hardship_amount,hardship_length,hardship_dpd,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,settlement_amount,settlement_percentage,settlement_term
count,0.0,2.260668e+06,2.260668e+06,2.260668e+06,2.260668e+06,2.260668e+06,2.260664e+06,2.258957e+06,2.260639e+06,2.260668e+06,...,10917.0,10917.000000,10917.0,10917.000000,8651.000000,10917.000000,10917.000000,34246.000000,34246.000000,34246.000000
mean,NaN,1.504693e+04,1.504166e+04,1.502344e+04,1.309283e+01,4.458068e+02,7.799243e+04,1.882420e+01,3.068792e-01,6.985882e+02,...,3.0,155.045981,3.0,13.743886,454.798089,11636.883942,193.994321,5010.664267,47.780365,13.191322
std,NaN,9.190245e+03,9.188413e+03,9.192332e+03,4.832138e+00,2.671735e+02,1.126962e+05,1.418333e+01,8.672303e-01,3.301038e+01,...,0.0,129.040594,0.0,9.671178,375.385500,7625.988281,198.629496,3693.122590,7.311822,8.159980
min,NaN,5.000000e+02,5.000000e+02,0.000000e+00,5.310000e+00,4.930000e+00,0.000000e+00,-1.000000e+00,0.000000e+00,6.100000e+02,...,3.0,0.640000,3.0,0.000000,1.920000,55.730000,0.010000,44.210000,0.200000,0.000000
25%,NaN,8.000000e+03,8.000000e+03,8.000000e+03,9.490000e+00,2.516500e+02,4.600000e+04,1.189000e+01,0.000000e+00,6.750000e+02,...,3.0,59.440000,3.0,5.000000,175.230000,5627.000000,44.440000,2208.000000,45.000000,6.000000
50%,NaN,1.290000e+04,1.287500e+04,1.280000e+04,1.262000e+01,3.779900e+02,6.500000e+04,1.784000e+01,0.000000e+00,6.900000e+02,...,3.0,119.140000,3.0,15.000000,352.770000,10028.390000,133.160000,4146.110000,45.000000,14.000000
75%,NaN,2.000000e+04,2.000000e+04,2.000000e+04,1.599000e+01,5.933200e+02,9.300000e+04,2.449000e+01,0.000000e+00,7.150000e+02,...,3.0,213.260000,3.0,22.000000,620.175000,16151.890000,284.190000,6850.172500,50.000000,18.000000
max,NaN,4.000000e+04,4.000000e+04,4.000000e+04,3.099000e+01,1.719830e+03,1.100000e+08,9.990000e+02,5.800000e+01,8.450000e+02,...,3.0,943.940000,3.0,37.000000,2680.890000,40306.410000,1407.860000,33601.000000,521.350000,181.000000


## Target Variable: Loan Status


In [9]:
# Check target variable distribution
print("Loan Status Distribution:")
print("=" * 50)
status_counts = df["loan_status"].value_counts()
status_pct = df["loan_status"].value_counts(normalize=True) * 100

for status in status_counts.index:
    print(f"{status:30} {status_counts[status]:>10,} ({status_pct[status]:>5.1f}%)")

Loan Status Distribution:
Fully Paid                      1,076,751 ( 47.6%)
Current                           878,317 ( 38.9%)
Charged Off                       268,559 ( 11.9%)
Late (31-120 days)                 21,467 (  0.9%)
In Grace Period                     8,436 (  0.4%)
Late (16-30 days)                   4,349 (  0.2%)
Does not meet the credit policy. Status:Fully Paid      1,988 (  0.1%)
Does not meet the credit policy. Status:Charged Off        761 (  0.0%)
Default                                40 (  0.0%)


In [10]:
# For binary classification, we'll use:
# - Fully Paid = 0 (good loan)
# - Charged Off = 1 (default)
# We'll filter out ambiguous statuses in the cleaning notebook

binary_statuses = ["Fully Paid", "Charged Off"]
binary_count = df[df["loan_status"].isin(binary_statuses)].shape[0]
print(f"\nRows with clear outcomes (Fully Paid/Charged Off): {binary_count:,}")
print(f"Percentage of total: {binary_count / len(df) * 100:.1f}%")


Rows with clear outcomes (Fully Paid/Charged Off): 1,345,310
Percentage of total: 59.5%


## Key Features Preview


In [11]:
# Key features we'll use
key_features = [
    "loan_amnt",
    "term",
    "int_rate",
    "installment",
    "grade",
    "sub_grade",
    "emp_length",
    "home_ownership",
    "annual_inc",
    "verification_status",
    "purpose",
    "dti",
    "open_acc",
    "revol_bal",
    "revol_util",
    "total_acc",
]

existing_features = [f for f in key_features if f in df.columns]
print(f"Key features available: {len(existing_features)}/{len(key_features)}")
print("\nPreview of key features:")
df[existing_features].head()

Key features available: 16/16

Preview of key features:


,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,purpose,dti,open_acc,revol_bal,revol_util,total_acc
0,3600.0,36 months,13.99,123.03,C,C4,10+ years,MORTGAGE,55000.0,Not Verified,debt_consolidation,5.91,7.0,2765.0,29.7,13.0
1,24700.0,36 months,11.99,820.28,C,C1,10+ years,MORTGAGE,65000.0,Not Verified,small_business,16.06,22.0,21470.0,19.2,38.0
2,20000.0,60 months,10.78,432.66,B,B4,10+ years,MORTGAGE,63000.0,Not Verified,home_improvement,10.78,6.0,7869.0,56.2,18.0
3,35000.0,60 months,14.85,829.90,C,C5,10+ years,MORTGAGE,110000.0,Source Verified,debt_consolidation,17.06,13.0,7802.0,11.6,17.0
4,10400.0,60 months,22.45,289.91,F,F1,3 years,MORTGAGE,104433.0,Source Verified,major_purchase,25.37,12.0,21929.0,64.5,35.0


In [12]:
# Missing values in key features
print("Missing values in key features:")
print("=" * 50)
missing = df[existing_features].isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)
missing_df = pd.DataFrame({"Missing": missing, "Pct": missing_pct})
print(missing_df[missing_df["Missing"] > 0].sort_values("Pct", ascending=False))

Missing values in key features:
                     Missing   Pct
emp_length            146940  6.50
dti                     1744  0.08
revol_util              1835  0.08
loan_amnt                 33  0.00
term                      33  0.00
int_rate                  33  0.00
installment               33  0.00
grade                     33  0.00
sub_grade                 33  0.00
home_ownership            33  0.00
annual_inc                37  0.00
verification_status       33  0.00
purpose                   33  0.00
open_acc                  62  0.00
revol_bal                 33  0.00
total_acc                 62  0.00


## Save Raw Data

Save as parquet for faster loading in subsequent notebooks.


In [13]:
# Save as parquet
output_path = DATA_DIR / "lending_club_raw.parquet"
df.to_parquet(output_path, index=False)

file_size_mb = os.path.getsize(output_path) / (1024 * 1024)
print(f"Saved to: {output_path}")
print(f"File size: {file_size_mb:.1f} MB")

Saved to: ../data/raw/lending_club_raw.parquet
File size: 335.7 MB


In [14]:
# Verify saved data
df_verify = pd.read_parquet(output_path)
print(f"Verification: {df_verify.shape} == {df.shape} ? {df_verify.shape == df.shape}")

Verification: (2260701, 151) == (2260701, 151) ? True


## Next Steps

Proceed to `02_data_cleaning.ipynb` to:

- Filter to binary outcomes
- Handle missing values
- Remove data leakage columns
